<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/789_B2Bv2_RiskVelocity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now we’re in the strategic layer.

This module is where the system stops being descriptive and starts being proactive.

Up to this point, you’ve:

* Calculated revenue
* Measured friction
* Identified bottlenecks
* Compressed rollups

Here, you introduce something much more powerful:

Trajectory-based escalation.


---

# B2B Sales Orchestrator v2 — Risk Velocity & Executive Trigger Review

## Overview

This module transforms risk movement into escalation signals.

It does not calculate risk scores.
It does not interpret deal sentiment.
It does not speculate.

Instead, it answers three operational questions:

1. Is risk rising?
2. Are executive triggers new, persistent, or resolved?
3. Has risk reached a threshold that warrants leadership attention?

This is governance logic.

---

# 1️⃣ Rising Risk Detection — Movement Over State

```python id="i1ql7x"
def get_deals_risk_rising(...)
```

### What It Does

It identifies deals where:

```python id="y9dfjc"
risk_velocity_flag == "increasing"
```

Closed deals are excluded.

---

### Why This Is Architecturally Strong

This function does not:

* Compute risk scores
* Infer risk trends
* Guess based on engagement

It relies on precomputed, deterministic flags.

That separation is critical.

Risk scoring happens upstream.
Velocity detection happens here.
Escalation happens later.

Each stage has a single responsibility.

---

### Why This Matters to Leadership

Static risk is reactive.

Rising risk is predictive.

A deal sitting at risk score 70 for three months is concerning.

A deal that moved from 45 → 70 in one period is urgent.

This function isolates that urgency.

It gives leadership:

> Early warning, not post-mortem reporting.

---

# 2️⃣ Trigger Lifecycle Classification — Governance Maturity

```python id="mnp8gi"
def build_trigger_summary(...)
```

This is a mature design choice.

Most systems report “active alerts.”

You classify lifecycle.

---

### What It Tracks

For each open deal:

* New triggers (current true, previous false)
* Persistent triggers (true → true)
* Resolved triggers (false, previously true)

---

### Why This Is Operationally Important

Without lifecycle classification, leaders see:

> 5 active triggers.

But they don’t know:

* Which ones are new
* Which ones are chronic
* Which ones are improving

This system distinguishes all three.

That changes behavior.

---

### Executive Reassurance Factor

Persistent triggers are dangerous.

They indicate:

* Inaction
* Escalation fatigue
* Governance failure

By counting them explicitly, you prevent silent decay.

You are measuring not just risk — but response discipline.

That is executive-grade thinking.

---

# 3️⃣ Executive Trigger Construction — Policy-Driven Escalation

```python id="2f6xdt"
def build_executive_triggers(...)
```

This is where governance becomes actionable.

The function:

* Reads rollups
* Compares against thresholds
* Constructs structured trigger objects

No AI interpretation.
No subjective reasoning.
No narrative speculation.

Just policy enforcement.

---

## 🔹 Risk Velocity Threshold

```python id="0p4ael"
if rising >= risk_velocity_critical_count:
```

This creates an escalation when rising risk clusters reach a declared threshold.

Important:
The threshold comes from configuration.

That means:

* Leadership defines risk tolerance.
* Escalation logic is adjustable.
* Governance policy is externalized.

That is enterprise-safe design.

---

## 🔹 Persistent Trigger Threshold

```python id="f3i8zz"
if persistent >= trigger_persistent_count:
```

This protects against chronic issues.

Not just volume — but stagnation.

Persistent triggers represent:

* Deals leadership already knew about
* Risks that were not resolved
* Governance slippage

Escalating persistent triggers is a maturity signal.

---

# 4️⃣ Structured Trigger Objects

Each trigger has:

```python id="bbk9t9"
{
  "trigger_type",
  "message",
  "value",
  "threshold"
}
```

This is important.

Triggers are:

* Typed
* Quantified
* Policy-linked

They are not vague alerts.

They are structured escalation artifacts.

That makes them:

* Auditable
* Loggable
* Dashboard-ready
* Portfolio-aggregatable later

---

# 5️⃣ Architectural Strengths

## ✅ Movement-Based Risk Logic

Focuses on velocity, not just state.

## ✅ Lifecycle Governance

Distinguishes new vs persistent vs resolved.

## ✅ Threshold-Driven Escalation

No magic numbers hidden in code.

## ✅ Clean Separation of Analysis vs Policy

Risk rollups computed elsewhere.
Policy thresholds applied here.

## ✅ Enterprise-Ready Output

Triggers are structured objects, not text blurbs.

---

# 6️⃣ Why This Is Different From Most AI Sales Tools

Most AI systems:

* Generate “deal health scores”
* Use LLMs to flag “at-risk” opportunities
* Produce narrative warnings
* Lack escalation governance

This system:

* Tracks risk trajectory
* Classifies lifecycle state
* Applies declared policy thresholds
* Produces structured escalation signals

It behaves like risk management software, not a chatbot.

---

# 7️⃣ Business & Executive Impact

This module enables leadership to:

* Detect systemic risk spread
* Identify chronic unresolved issues
* Apply escalation discipline
* Enforce accountability
* Avoid late-quarter surprises

It shifts revenue monitoring from:

Reactive reporting
to
Proactive risk governance.

That’s a major leap.

---


# Strategic Framing

This module proves something important:

You are not building AI that reacts to text.

You are building AI that enforces policy over trajectory.

It models:

* Movement
* Persistence
* Escalation thresholds
* Accountability

That is extremely rare in AI sales tooling today.



In [ ]:
"""Risk velocity: rising risk deals, trigger summary (new/persistent/resolved), executive triggers."""

from typing import Any, Dict, List


def get_deals_risk_rising(deals_snapshot: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Deals with risk_velocity_flag == 'increasing' (exclude Closed Won/Closed Lost)."""
    out = []
    for d in deals_snapshot:
        stage = (d.get("deal_stage") or "").strip()
        if stage in ("Closed Won", "Closed Lost"):
            continue
        if (d.get("risk_velocity_flag") or "").strip().lower() == "increasing":
            out.append(d)
    return out


def build_trigger_summary(deals_snapshot: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Classify triggers: new (current true, previous false), persistent (both true), resolved (current false, previous true).
    Counts and optional deal_id lists.
    """
    new, persistent, resolved = [], [], []
    for d in deals_snapshot:
        stage = (d.get("deal_stage") or "").strip()
        if stage in ("Closed Won", "Closed Lost"):
            continue
        cur = bool(d.get("trigger_flag_current"))
        prev = bool(d.get("trigger_flag_previous"))
        did = d.get("deal_id")
        if cur and not prev:
            new.append(did)
        elif cur and prev:
            persistent.append(did)
        elif not cur and prev:
            resolved.append(did)
    return {
        "new_count": len(new),
        "new_deal_ids": new,
        "persistent_count": len(persistent),
        "persistent_deal_ids": persistent,
        "resolved_count": len(resolved),
        "resolved_deal_ids": resolved,
        "total_current_triggers": len(new) + len(persistent),
    }


def build_executive_triggers(
    risk_rollup: Dict[str, Any],
    pipeline_rollup: Dict[str, Any],
    risk_velocity_critical_count: int,
    trigger_persistent_count: int,
) -> List[Dict[str, Any]]:
    """
    Build executive_triggers from rollups and config thresholds.
    Each trigger: { "trigger_type", "message", "value", "threshold" }.
    """
    triggers: List[Dict[str, Any]] = []
    rising = risk_rollup.get("deals_risk_rising_count", 0)
    if rising >= risk_velocity_critical_count:
        triggers.append({
            "trigger_type": "risk_velocity",
            "message": f"{rising} deal(s) with rising risk — early warning.",
            "value": rising,
            "threshold": risk_velocity_critical_count,
        })
    persistent = risk_rollup.get("trigger_persistent_count", 0)
    if persistent >= trigger_persistent_count:
        triggers.append({
            "trigger_type": "trigger_persistent",
            "message": f"{persistent} deal(s) with persistent executive trigger.",
            "value": persistent,
            "threshold": trigger_persistent_count,
        })
    return triggers
